In [1]:
import path
import libs.checker as checker
from control import *
from z3 import *
from scipy.signal import lti, step, impulse
import scipy.signal as sig
from matplotlib import pyplot as p

This example is borrowed and adapted from [1], design problem 5.3 on Pg. 360. The following transfer function represents an active suspension system for modern cars to provide comfortable firm ride:
$$ H(s) = \frac{K}{s^2 + q s + K}$$

[1] Richard C.. Dorf, and Robert H. Bishop. Modern control systems. Pearson Prentice Hall, 2016. Pg. 360

The step input to the system could manifest a disturnbance, or a tracking to a reference. In these contexts, the main job of an active suspension system is to quickly settle to the final value. We therefore have the following requirement: with $q=17$ and $K$ being free variable, does the design always have the settling time (0.01% criterian) $T_s \le 0.5$?

In [2]:
# Define Constants Symbolically
K = Real('K')
zeta =  Real('zeta')
omega_n = Real('omega_n')
r = Real('r')
q = 17
wn_square = omega_n * omega_n

f0 = omega_n * omega_n == K
f1 = q == 2 * zeta * omega_n
context = And(f0,f1)

In [3]:

# Setup
c = checker.Checker(zeta,omega_n)

# Safety
f = c.factory
safety = f.settle < 0.5

# PO generation
c.auto_pog(safety, context)

In [4]:
# Numerical Static Checking in Z3 SMT solver
c.result()

static checking result =  sat
static checking model =  [peak = 1,
 t = -0.5,
 temp = 0.5625,
 y = Store(Store(Store(K(Real, -0.5), 1, 1), 0, 0.1),
           0.0377107180?,
           0.9),
 zeta = -0.5483870967?,
 t2 = 0,
 omega_n = -15.5,
 overshoot = 0,
 r = 1,
 steady = 1,
 input = 1,
 rise = 0.0377107180?,
 settle = 0.5411764705?,
 dc = 1,
 K = 240.25,
 /0 = [(240.25, 240.25) -> 1,
       (-0.5845161290?, -15.5) -> 0.0377107180?,
       else -> -0.3601433691?]]


A counterexample is found that at the given K, we have settling time of 0.54 which violates the requirement.